In [ ]:
#  Code ref: https://ritscm.regeneron.com/projects/DSEAIM/repos/secureaccess/browse/securekeyapp.py

In [1]:
!pip install --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org --upgrade pip

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/15/aa/3f4c7bcee2057a76562a5b33ecbd199be08cdb4443a02e26bd2c3cf6fc39/pip-23.3.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1


In [2]:
!pip install  --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host files.pythonhosted.org \
msal \
retry \
simplejson \
openai \
pymilvus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 1.0 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 801.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 953.0 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 1.2 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.9/174.9 kB 1.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 810.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 730.8 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13

In [3]:
import os
import openai
import requests
import simplejson as json
from retry import retry
from msal import PublicClientApplication, ConfidentialClientApplication, ClientApplication
import time

In [4]:
@retry (tries=3, delay=2)
def getapikey():
    # These are the AZURE parameters needed by the client application
    # In this scheme, the user does not have to worry about the api key, all of that is handled
    # at the AZURE Back End.   The API key is rotated hourly.
    # We can handle these as Kuberbetes secrets, or as environment variables.
    client_id = os.getenv("AZURE_CLIENT_ID")
    tenant_id = os.getenv("AZURE_TENANT_ID")
    endpoint = os.getenv("AZURE_ENDPOINT")

    scopes = [os.getenv("AZURE_APPLICATION_SCOPE")]

    app = ClientApplication(
        client_id=client_id,
        authority="https://login.microsoftonline.com/" + tenant_id
    )


    acquire_tokens_result = app.acquire_token_by_username_password(username=os.getenv("SVC_ACCOUNT"),
                                                                   password=os.getenv("SVC_PASSWORD"),
                                                                   scopes=scopes)
    if 'error' in acquire_tokens_result:
        print("Error: " + acquire_tokens_result['error'])
        print("Description: " + acquire_tokens_result['error_description'])
        return 2
    else:
        header_token = {"Authorization": "Bearer {}".format(acquire_tokens_result['access_token'])}
        rt = requests.post(url=endpoint, headers=header_token, data=b'{"key":"openaikey2"}')
        return rt.json()

In [5]:
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
# We are dynamically getting the key from AZURE>   Access is based on the service account/ad group combination
openai.api_key = getapikey()
print(openai.api_key)

c933dde5c53b4ad1bf4dce345c678fe7


In [9]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key = getapikey(),  
  api_version = os.getenv("OPENAI_API_VERSION"),
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

protein = "{\"_id\":\"582b467fbd364f2b2a00009c\",\"_type\":\"Protein\",\"associated_plasmid_regn_names\":[],\"created_at\":\"2016-11-15T17:31:43.154Z\",\"created_by\":\"kristin.hudson\",\"eln_ref\":\"https://researcheln.regeneron.regn.com:8443/EWorkbookWebApp/#entity/displayEntity?entityId=e4cd7280ab5811e6a49840a8f01e721c\\u0026r=y\",\"linked_pes_prgns\":null,\"pes_created_at\":null,\"pes_created_by\":null,\"project_numbers\":[],\"protein_chain_ids\":[\"582b45c777ab416e41000031\",\"582b462ebd364f2b2a00009b\"],\"protein_sources\":[{\"_id\":\"5f7e341e7baad70118845ce8\",\"created_at\":\"2020-10-07T21:33:18.831Z\",\"regn_name\":\"PN22022\",\"updated_at\":\"2020-10-07T21:33:18.831Z\"}],\"protein_type\":{\"_id\":\"5d07a07a700f3e489862adc1\",\"chain_structure\":[2,2],\"chains\":4,\"created_at\":\"2016-11-09T19:27:50.386Z\",\"name\":\"Standard mAb\",\"protein_category\":\"Common Antibody \\u0026 Antibody-like Formats\",\"protein_id\":null,\"svg\":\"data:image/svg+xml;base64, IDxzdmcgdmVyc2lvbj0iMS4xIiBpZD0iTGF5ZXJfMSIgeG1sbnM9Imh0dHA6\\nLy93d3cudzMub3JnLzIwMDAvc3ZnIiB4bWxuczp4bGluaz0iaHR0cDovL3d3\\ndy53My5vcmcvMTk5OS94bGluayIgeD0iMHB4IiB5PSIwcHgiIHdpZHRoPSIz\\nODhweCIgaGVpZ2h0PSIzNTkuMjI3cHgiIHZpZXdCb3g9IjAgMCAzODggMzU5\\nLjIyNyIgZW5hYmxlLWJhY2tncm91bmQ9Im5ldyAwIDAgMzg4IDM1OS4yMjci\\nIHhtbDpzcGFjZT0icHJlc2VydmUiPiA8Zz4gPGc+IDxyZWN0IHg9IjE5Ni41\\nIiB5PSIxNTYuOTc3IiBmaWxsPSIjQ0MwMDAwIiB3aWR0aD0iMjYuMTU3IiBo\\nZWlnaHQ9IjE4NC41Ii8+IDwvZz4gPC9nPiA8Zz4gPGc+IDxyZWN0IHg9IjE2\\nNy41IiB5PSIxNTYuOTc3IiBmaWxsPSIjQ0MwMDAwIiB3aWR0aD0iMjYuMTU3\\nIiBoZWlnaHQ9IjE4NC41Ii8+IDwvZz4gPC9nPiA8cmVjdCB4PSIzNTEuNSIg\\neT0iMzI3LjcyNyIgZmlsbD0iI0NDMDAwMSIgd2lkdGg9IjEiIGhlaWdodD0i\\nMCIvPiA8Zz4gPHJlY3QgeD0iMjY1LjU2MSIgeT0iLTkuNzUyIiB0cmFuc2Zv\\ncm09Im1hdHJpeCgwLjcwNzEgMC43MDcxIC0wLjcwNzEgMC43MDcxIDEzOS45\\nNDY3IC0xNzIuODY1MikiIGZpbGw9IiNDQzAwMDAiIHdpZHRoPSIyNi4xNTci\\nIGhlaWdodD0iMTg0LjUiLz4gPC9nPiA8Zz4gPHJlY3QgeD0iOTguNDM5IiB5\\nPSItOS43NTIiIHRyYW5zZm9ybT0ibWF0cml4KC0wLjcwNzEgMC43MDcxIC0w\\nLjcwNzEgLTAuNzA3MSAyNDguNzA3NiA2MS45ODA5KSIgZmlsbD0iI0NDMDAw\\nMCIgd2lkdGg9IjI2LjE1NyIgaGVpZ2h0PSIxODQuNSIvPiA8L2c+IDxnPiA8\\ncmVjdCB4PSI3Ni43OTMiIHk9IjExLjc4NCIgdHJhbnNmb3JtPSJtYXRyaXgo\\nLTAuNzA3MSAwLjcwNzEgLTAuNzA3MSAtMC43MDcxIDIyNi43OTU2IDExMy41\\nOTgpIiBmaWxsPSIjMDAwMEZGIiB3aWR0aD0iMjYuMTU3IiBoZWlnaHQ9IjE4\\nMy45NyIvPiA8L2c+IDxnPiA8cmVjdCB4PSIyODguMjQ1IiB5PSIxMS43ODUi\\nIHRyYW5zZm9ybT0ibWF0cml4KC0wLjcwNzEgLTAuNzA3MSAwLjcwNzEgLTAu\\nNzA3MSA0NDEuMDE2MSAzOTAuMjEzOSkiIGZpbGw9IiMwMDAwRkYiIHdpZHRo\\nPSIyNi4xNTciIGhlaWdodD0iMTgzLjk3Ii8+IDwvZz4gPC9zdmc+IA==\\n\",\"unique_chains\":2,\"updated_at\":\"2016-11-09T19:27:50.386Z\"},\"purposes\":[\"Isotype Control\"],\"pushed_to_lims\":true,\"regn_name\":\"REGN5012\",\"regn_number\":5012,\"resolved_at\":\"2016-11-15T17:31:43.154Z\",\"resolved_by\":\"kristin.hudson\",\"synonyms\":[{\"_id\":\"5b6020e3700f3e449e654a63\",\"type\":\"abpid\",\"value\":\"H4H22022N\"}],\"updated_at\":\"2023-04-21T09:45:56.211Z\"}"

response = client.embeddings.create(
    input = protein,
    model= "RegnADA002"
)

print(response.model_dump_json(indent=2))

{
  "data": [
    {
      "embedding": [
        -0.01193129550665617,
        -0.009233196265995502,
        -0.02271013706922531,
        -0.02269657887518406,
        -0.018520625308156013,
        0.0011066955048590899,
        0.0034387214109301567,
        0.012846480123698711,
        -0.025448912754654884,
        -0.00888745952397585,
        0.0012032983358949423,
        -0.01529375184327364,
        0.009829760529100895,
        0.00666389986872673,
        0.005724988412111998,
        0.041868001222610474,
        0.02691320702433586,
        -0.03630910441279411,
        0.0027150476817041636,
        0.010643258690834045,
        -0.017164796590805054,
        0.006436798721551895,
        -0.031590819358825684,
        -0.023659218102693558,
        0.009185741655528545,
        0.00738926837220788,
        0.012351603247225285,
        -0.023306701332330704,
        0.0030472257640212774,
        -0.026425108313560486,
        -0.0021489891223609447,
        0.0009617

In [7]:
from pymilvus import connections, db
conn = connections.connect(host="standalone", port=19530, db_name="default")
db.list_database()

['default']